In [206]:
import numpy as np

from utils import gaussjordan

from LinearCode import LinearCode


class QC_LDPC(LinearCode):
    """
    Quasi-Cyclic LDPC code representation (extends LinearCode)

    ...

    Methods
    -------
    from_params(n, p, w)
        Init QC-LDPC by length, circulant size and code weight

    _get_circulant_block(polynom)
        Get circulant (p, p) for given vector of size p
        
    """

    def __init__(self, G, H):
        super().__init__(G, H)
        
    @classmethod
    def from_params(cls, n, p, w):
        assert n % p == 0, "p must be delimeter of n"
        
        n0 = n // p
        assert w > 2*n0, "not enough code weight"
        
        fine = False
        
        while not fine:
            blocks = []
            inverse_block = None
            inverse_block_position = None
            
            vector = [1 for _ in range(w)] + [0 for _  in range(n - w)]
            vector = np.array(vector, dtype=int)
            np.random.shuffle(vector)
            
            for i in range(n0):
                circ = vector[i*p:(i+1)*p]
                
                if sum(circ) < 2:
                    inverse_block = None
                    break
        
                block = QC_LDPC._get_circulant_block(circ)
                blocks.append(block)
                    
                A, P = gaussjordan(block, True)
                A = np.array(A, dtype=int)
                P = np.array(P, dtype=int)
                
                if (A == np.eye(p, dtype=int)).all():
                    inverse_block_position = i
                    inverse_block = P
                
            # continue only if inverse circulant found
            fine = True if inverse_block is not None else False
        
        # put inverse block on last position
        blocks[inverse_block_position], blocks[n0-1] = blocks[n0-1], blocks[inverse_block_position]
        H = np.concatenate(blocks, axis=1)

        for i in range(n0):
            blocks[i] = blocks[i] @ inverse_block % 2
            blocks[i] = blocks[i].T            

        Ht = np.concatenate(blocks[:n0-1], axis=0)
        G = np.concatenate((np.eye(Ht.shape[0], dtype=int), Ht), axis=1)
        
        assert (G @ H.T % 2 == 0).all(), "G is not correspond to H"
        
        return cls(G, H)
    
    @staticmethod
    def _get_circulant_block(polynom):
        N = len(polynom)
        block = np.empty((N, N), dtype=int)
        
        for i in range(N):
            block[i] = np.roll(polynom, i)
            
        return block



# EXAMPLE USAGE:

n = 16
p = 8
w = 7

qc_ldpc = QC_LDPC.from_params(n, p, w)
print(qc_ldpc.getG())
print(qc_ldpc.getG().shape)

[[1 0 0 0 0 0 0 0 1 0 0 1 1 0 1 1]
 [0 1 0 0 0 0 0 0 1 1 0 0 1 1 0 1]
 [0 0 1 0 0 0 0 0 1 1 1 0 0 1 1 0]
 [0 0 0 1 0 0 0 0 0 1 1 1 0 0 1 1]
 [0 0 0 0 1 0 0 0 1 0 1 1 1 0 0 1]
 [0 0 0 0 0 1 0 0 1 1 0 1 1 1 0 0]
 [0 0 0 0 0 0 1 0 0 1 1 0 1 1 1 0]
 [0 0 0 0 0 0 0 1 0 0 1 1 0 1 1 1]]
(8, 16)


In [207]:
print(qc_ldpc.getH())
print(qc_ldpc.getH().shape)

[[0 1 0 0 1 0 1 0 1 0 1 1 0 0 0 0]
 [0 0 1 0 0 1 0 1 0 1 0 1 1 0 0 0]
 [1 0 0 1 0 0 1 0 0 0 1 0 1 1 0 0]
 [0 1 0 0 1 0 0 1 0 0 0 1 0 1 1 0]
 [1 0 1 0 0 1 0 0 0 0 0 0 1 0 1 1]
 [0 1 0 1 0 0 1 0 1 0 0 0 0 1 0 1]
 [0 0 1 0 1 0 0 1 1 1 0 0 0 0 1 0]
 [1 0 0 1 0 1 0 0 0 1 1 0 0 0 0 1]]
(8, 16)


In [208]:
word = np.random.randint(2, size=qc_ldpc.getG().shape[0])

In [209]:
word

array([1, 0, 1, 0, 1, 0, 0, 1])

In [210]:
print(len(word))

8


In [211]:
encoded = qc_ldpc.encode(word)
print(encoded)
print(len(encoded))

[1 0 1 0 1 0 0 1 1 1 1 1 0 0 1 1]
16


In [212]:
# error vector size n with t or less errors
errors_num = 2
e = [1 for _ in range(errors_num)] + [0 for _  in range(n - errors_num)]
e = np.array(e)
np.random.shuffle(e)
print(e)
print(len(e))

[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
16


In [213]:
corrupted = (encoded + e) % 2
print(corrupted)
print(len(corrupted))

[1 0 1 0 1 0 0 1 0 1 1 1 0 0 1 0]
16


In [214]:
decoded = qc_ldpc.decode(np.copy(corrupted))
decoded = qc_ldpc.get_message(decoded)
print(decoded)
print(len(decoded))

[1 0 0 0 0 1 1 0]
8


In [215]:
word

array([1, 0, 1, 0, 1, 0, 0, 1])

In [216]:
try:
    assert (decoded == word).all()
except AssertionError:
    print("The secret message must be forwarded")

The secret message must be forwarded


In [ ]:
from google.colab import drive
import os
import shutil
drive.mount('/content/gdrive')

In [ ]:
os.chdir('/content/gdrive/My Drive/Animal Species')

In [149]:
ls

 Le volume dans le lecteur C n’a pas de nom.
 Le numéro de série du volume est 3C58-3F56

 Répertoire de C:\Users\Willy\Desktop\github\Animal Species

13/07/2024  09:17    <DIR>          .
13/07/2024  10:04    <DIR>          ..
11/07/2024  22:59    <DIR>          abalone
11/07/2024  22:59    <DIR>          Abalones
11/07/2024  22:59    <DIR>          Acadian
11/07/2024  22:59    <DIR>          Acoupa
11/07/2024  22:59    <DIR>          Aculeate
11/07/2024  22:59    <DIR>          Adanson's
11/07/2024  22:59    <DIR>          Adelie
11/07/2024  22:59    <DIR>          Aden
11/07/2024  22:59    <DIR>          Adonis
11/07/2024  22:59    <DIR>          Adriatic
11/07/2024  22:59    <DIR>          Adusta
11/07/2024  22:59    <DIR>          Aesop
11/07/2024  22:59    <DIR>          Africa-bighead
11/07/2024  22:59    <DIR>          African
11/07/2024  22:59    <DIR>          Agar
11/07/2024  22:59    <DIR>          Agate
11/07/2024  22:59    <DIR>          Agujon
11/07/2024  22:59    <DIR> 

11/07/2024  22:59    <DIR>          Basket
11/07/2024  22:59    <DIR>          Basking
11/07/2024  22:59    <DIR>          bass
11/07/2024  22:59    <DIR>          Bastard
11/07/2024  22:59    <DIR>          Bat
11/07/2024  22:59    <DIR>          Batfishes
11/07/2024  22:59    <DIR>          bathing
11/07/2024  22:59    <DIR>          Bathyraja
11/07/2024  22:59    <DIR>          batissa
11/07/2024  22:59    <DIR>          Batoid
11/07/2024  22:59    <DIR>          Batwing
11/07/2024  22:59    <DIR>          Bay
11/07/2024  22:59    <DIR>          beach
11/07/2024  22:59    <DIR>          Beadlet
11/07/2024  22:59    <DIR>          beaked
11/07/2024  22:59    <DIR>          Bean
11/07/2024  22:59    <DIR>          Bear
11/07/2024  22:59    <DIR>          Bearded
11/07/2024  22:59    <DIR>          beardfish
11/07/2024  22:59    <DIR>          Beautiful
11/07/2024  22:59    <DIR>          Beautifully-banded
11/07/2024  22:59    <DIR>          Beebe's
11/07/2024  22:59    <DIR>         

11/07/2024  22:59    <DIR>          Sakura
11/07/2024  22:59    <DIR>          Salamander
11/07/2024  22:59    <DIR>          Saldanha
11/07/2024  22:59    <DIR>          Salema
11/07/2024  22:59    <DIR>          salmon
11/07/2024  22:59    <DIR>          Salmonoids
11/07/2024  22:59    <DIR>          salmons
11/07/2024  22:59    <DIR>          Salps
11/07/2024  22:59    <DIR>          Salvin's
11/07/2024  22:59    <DIR>          Samar
11/07/2024  22:59    <DIR>          Sammara
11/07/2024  22:59    <DIR>          Samoan
11/07/2024  22:59    <DIR>          San
11/07/2024  22:59    <DIR>          Sand
11/07/2024  22:59    <DIR>          Sandbar
11/07/2024  22:59    <DIR>          Sandbeach
11/07/2024  22:59    <DIR>          Sandbird
11/07/2024  22:59    <DIR>          sanddab
11/07/2024  22:59    <DIR>          sandeel
11/07/2024  22:59    <DIR>          sand-eel
11/07/2024  22:59    <DIR>          sandeels
11/07/2024  22:59    <DIR>          Sandeels(
11/07/2024  22:59    <DIR>      

11/07/2024  22:59    <DIR>          Suco
11/07/2024  22:59    <DIR>          Suez
11/07/2024  22:59    <DIR>          sugar
11/07/2024  22:59    <DIR>          Sulcate
11/07/2024  22:59    <DIR>          Summan
11/07/2024  22:59    <DIR>          Summer
11/07/2024  22:59    <DIR>          Sunburst
11/07/2024  22:59    <DIR>          Sunda
11/07/2024  22:59    <DIR>          sundial
11/07/2024  22:59    <DIR>          sunetta
11/07/2024  22:59    <DIR>          sunfish
11/07/2024  22:59    <DIR>          sunfishes
11/07/2024  22:59    <DIR>          sunset
11/07/2024  22:59    <DIR>          Suratan
11/07/2024  22:59    <DIR>          Surf
11/07/2024  22:59    <DIR>          Surge
11/07/2024  22:59    <DIR>          surgeonfish
11/07/2024  22:59    <DIR>          Surgeonfishes
11/07/2024  22:59    <DIR>          Surinam
11/07/2024  22:59    <DIR>          Suriname
11/07/2024  22:59    <DIR>          Surmullet
11/07/2024  22:59    <DIR>          Surmullets(
11/07/2024  22:59    <DIR>    

In [144]:
value = 0 
dico = {}
for filename in os.listdir('/content/gdrive/My Drive/Animal Species'):
        if(os.path.isdir(filename)):
        dico[value]  = filename
        value = value + 1

In [145]:
 for i in dico.items():
    print(i)    

(0, 'AABI')
(1, 'AABID')
(2, 'AABIV')
(3, 'AALBERG')
(4, 'AAMARA')
(5, 'AARAB')
(6, 'AARNINK')
(7, 'AARON')
(8, 'AARONG')
(9, 'AARRAS')
(10, 'AATAR')
(11, 'AATIF')
(12, 'AATZ')
(13, 'AAZIZ')
(14, 'ABA')
(15, 'ABABOU')
(16, 'ABABSA')
(17, 'ABACHIN')
(18, 'ABAD')
(19, 'ABADA')
(20, 'ABADE')
(21, 'ABADI')
(22, 'ABADIA')
(23, 'ABADIANO')
(24, 'ABADIAS')
(25, 'ABADIE')
(26, 'ABADLI')
(27, 'ABADLIA')
(28, 'ABADO')
(29, 'ABADOU')
(30, 'ABAFFOUR')
(31, 'ABAFOUR')
(32, 'ABAIDIA')
(33, 'ABAILLE')
(34, 'ABAIR')
(35, 'ABAJO')
(36, 'ABALAIN')
(37, 'ABALAM')
(38, 'ABALEA')
(39, 'ABALEO')
(40, 'ABALI')
(41, 'ABALLAIN')
(42, 'ABALLEA')
(43, 'ABALO')
(44, 'ABANADES')
(45, 'ABANCOURT')
(46, 'ABANE')
(47, 'ABANES')
(48, 'ABANI')
(49, 'ABAOUI')
(50, 'ABAR')
(51, 'ABARCA')
(52, 'ABARE')
(53, 'ABARKAN')
(54, 'ABARKI')
(55, 'ABARNOU')
(56, 'ABARO')
(57, 'ABART')
(58, 'ABAS')
(59, 'ABASCAL')
(60, 'ABASQ')
(61, 'ABASSE')
(62, 'ABASSEUR')
(63, 'ABASSI')
(64, 'ABASSIE')
(65, 'ABASTADO')
(66, 'ABAT')
(67, 'ABATAN

(789, 'BARTMANN')
(790, 'BARTNICKI')
(791, 'BARTNIK')
(792, 'BARTOCCI')
(793, 'BARTOCCIONI')
(794, 'BARTOCHE')
(795, 'BARTOIS')
(796, 'BARTOLACCI')
(797, 'BARTOLETTI')
(798, 'BARTOLI')
(799, 'BARTOLIN')
(800, 'BARTOLINI')
(801, 'BARTOLL')
(802, 'BARTOLO')
(803, 'BARTOLOME')
(804, 'BARTOLOMEI')
(805, 'BARTOLOMEO')
(806, 'BARTOLOMEU')
(807, 'BARTOLOMUCCI')
(808, 'BARTOLONE')
(809, 'BARTOLONI')
(810, 'BARTOLOTTA')
(811, 'BARTOLOTTI')
(812, 'BARTOLOZZI')
(813, 'BARTOLUCCI')
(814, 'BARTON')
(815, 'BARTONICEK')
(816, 'BARTOS')
(817, 'BARTOSIK')
(818, 'BARTOSZ')
(819, 'BARTOSZCZYK')
(820, 'BARTOSZEK')
(821, 'BARTOSZEWSKI')
(822, 'BARTOT')
(823, 'BARTOU')
(824, 'BARTOUILH')
(825, 'BARTOUT')
(826, 'BARTOUX')
(827, 'BARTRA')
(828, 'BARTRE')
(829, 'BARTRINA')
(830, 'BARTRINGER')
(831, 'BARTSCH')
(832, 'BARTY')
(833, 'BARTYZEL')
(834, 'BARTZ')
(835, 'BARTZEN')
(836, 'BARU')
(837, 'BARUBE')
(838, 'BARUCCHI')
(839, 'BARUCCI')
(840, 'BARUCH')
(841, 'BARUCHE')
(842, 'BARUCHEL')
(843, 'BARUCHELLO')
(84

In [146]:
y = corrupted
print(y)
len(y)

[0 1 0 1 0 0 0 0 1 1 1 0 0 1 0 0 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 1 1 1 0 1
 1 1 1 0 1 1 1 0 0 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 1 0 1
 1 0 0 0 1 0 1 1 1 0 1 0 0 1 1 1 0 0 1 0 1 1 1 0 1 1 1 0 1 0 1 0 1 0 1 1 1
 0 1 1 0 0 1 0 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0 1 0 1 0 1 0 0 0 1 1 1 1 0 1
 1 0 0 1 0 1 1 1 0 0 0 1 1 1 0 1 1 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0
 0 0 0 1 0 1 1 1 1 1 1 1 1 1 0 0 0 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 0 1 0 0 1
 1 0 0 0]


226

In [148]:
lis = []
i = 0

while i < len(y):
    print(dico[i])
    count = 0
    for filename in os.listdir('/content/gdrive/My Drive/Animal Species/' + dico[i]):
        print(filename)
        count  = count + 1
    lis.append(count)
    print("-- nombre de fichiers  = ", lis[i],"--")
    print("\n")
    i = i + 1

AABI
business_89.txt
business_90.txt
-- nombre de fichiers  =  2 --


AABID
business_87.txt
business_88.txt
-- nombre de fichiers  =  2 --


AABIV
-- nombre de fichiers  =  0 --


AALBERG
business_85.txt
business_86.txt
-- nombre de fichiers  =  2 --


AAMARA
business_83.txt
business_84.txt
-- nombre de fichiers  =  2 --


AARAB
business_81.txt
business_82.txt
-- nombre de fichiers  =  2 --


AARNINK
business_79.txt
business_80.txt
-- nombre de fichiers  =  2 --


AARON
business_77.txt
business_78.txt
-- nombre de fichiers  =  2 --


AARONG
-- nombre de fichiers  =  0 --


AARRAS
business_29.txt
business_30.txt
-- nombre de fichiers  =  2 --


AATAR
business_75.txt
business_76.txt
-- nombre de fichiers  =  2 --


AATIF
business_34.txt
business_35.txt
-- nombre de fichiers  =  2 --


AATZ
business_23.txt
business_24.txt
-- nombre de fichiers  =  2 --


AAZIZ
business_73.txt
business_74.txt
-- nombre de fichiers  =  2 --


ABA
business_25.txt
business_26.txt
-- nombre de fichiers  =  2 -

In [149]:
lis

[2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 4,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 6,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 8,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 2,
 2,
 2,
 2,
 6,
 6,
 4,
 6,
 4,
 6,
 4,
 4,
 8,
 2,
 4,
 4,
 2,
 4,
 4,
 4,
 2,
 4,
 4,
 4,
 6,
 4,
 6,
 6,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [150]:
import hashlib
import os


def hash_directory(path):
    digest = hashlib.sha256()

    for root, dirs, files in os.walk(path):
        for names in files:
            file_path = os.path.join(root, names)

            # Hash the path and add to the digest to account for empty files/directories
            digest.update(hashlib.sha1(file_path[len(path):].encode()).digest())

            # Per @pt12lol - if the goal is uniqueness over repeatability, this is an alternative method using 'hash'
            # digest.update(str(hash(file_path[len(path):])).encode())

            if os.path.isfile(file_path):
                with open(file_path, 'rb') as f_obj:
                    while True:
                        buf = f_obj.read(1024 * 1024)
                        if not buf:
                            break
                        digest.update(buf)

    return digest.hexdigest()

In [151]:
list_Hash1 = []
for i in range(len(y)):
    list_Hash1.append('')
list_Hash1

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',


In [152]:
for i in range(len(y)):
    path = '/content/gdrive/My Drive/Animal Species/' + dico[i]
    #print(path)
    list_Hash1[i] = hash_directory(path)
    print('--hash '+dico[i],'--',list_Hash1[i])

--hash AABI -- ddeab86c870b0614ba3eb793891d54fe232c4efb3121c9d5f6994f1483f79c3c
--hash AABID -- 8e6d88136ac756d00ed246c585aacbe629e01d2dd452f4d33c014f7605a9687c
--hash AABIV -- e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
--hash AALBERG -- 83e11e8eaa0d01c55616d2ad0dafa89f92fa8341924e1bbf461d0506c58cc661
--hash AAMARA -- c5348ccbb5811c81fc4af40f1ed797e980e15b82da1f5fddd56c301dd285abe8
--hash AARAB -- 79adfe0e9c3de126179c2c3ff2279f4bc2a6ecec366c7d7a8a2e780bc4391730
--hash AARNINK -- cc8c050946c63c044dad30ce38ca805d7b13b15a8c56da8bddd37ac003c8476b
--hash AARON -- ca820cb14d9294d7f943ba69ad50c3a2f0447bfef57ad7308aa268f87ec83d65
--hash AARONG -- e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
--hash AARRAS -- e58dd68e06f54ba8b1b932265e5465d9759a409024eea89d4c66494e266e6c02
--hash AATAR -- f0023d0b5a3f94154f63eb9fca8d4f5a47c254d711123e9d3cfe00eb40dda1f4
--hash AATIF -- 28f8c0179dd18b567738e76000223962fbbda23f47a432e1f20f69436361df99
--hash AATZ -- 6e7de77

In [153]:
y

array([0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 0], dtype=int32)

In [ ]:
liste = ['Balistidae', 'Belonidae', 'Enoploteuthidae', 'Cyprinidae', 'Nephropidae', 'Axiidae', 'Serranidae', 'Cardiidae', 'Scombridae', 'Sparidae', 'Acanthuridae', 'Limidae', 'Sergestidae', 'Acipenseridae', 'Acropomatidae', 'Holothuriidae', 'Squalidae', 'Octopodidae', 'Pectinidae', 'Serranidae']

In [ ]:
len(liste)

In [154]:
i = 0
while i < len(y):
    number = np.random.randint(0,20)
    file = '/'+liste[number]+'.txt'
    if(y[i] == 1):
        my_path = 'C:/Users/Willy/Desktop/github/Animal Species/' + dico[i]
        os.chdir(my_path)
        #print(my_path + file)
        if not os.path.exists(my_path + file):
            with open(my_path + file,'w') as fp:
                pass
        else:
            descriptor = open(my_path + file,'a')
            descriptor.write("Species")
            descriptor.close()
        #print('-- folder f'+str(j),' -- file0 -- ',filename)
    i = i + 1

In [155]:
lis = []
i = 0

while i < len(y):
    print(dico[i])
    count = 0
    for filename in os.listdir('/content/gdrive/My Drive/Animal Species/' + dico[i]):
        print(filename)
        count  = count + 1
    lis.append(count)
    print("-- nombre de fichiers  = ", lis[i],"--")
    print("\n")
    i = i + 1

AABI
business_89.txt
business_90.txt
-- nombre de fichiers  =  2 --


AABID
business_87.txt
business_88.txt
file.txt
-- nombre de fichiers  =  3 --


AABIV
-- nombre de fichiers  =  0 --


AALBERG
business_85.txt
business_86.txt
file.txt
-- nombre de fichiers  =  3 --


AAMARA
business_83.txt
business_84.txt
-- nombre de fichiers  =  2 --


AARAB
business_81.txt
business_82.txt
-- nombre de fichiers  =  2 --


AARNINK
business_79.txt
business_80.txt
-- nombre de fichiers  =  2 --


AARON
business_77.txt
business_78.txt
-- nombre de fichiers  =  2 --


AARONG
file.txt
-- nombre de fichiers  =  1 --


AARRAS
business_29.txt
business_30.txt
file.txt
-- nombre de fichiers  =  3 --


AATAR
business_75.txt
business_76.txt
file.txt
-- nombre de fichiers  =  3 --


AATIF
business_34.txt
business_35.txt
-- nombre de fichiers  =  2 --


AATZ
business_23.txt
business_24.txt
-- nombre de fichiers  =  2 --


AAZIZ
business_73.txt
business_74.txt
file.txt
-- nombre de fichiers  =  3 --


ABA
busines

In [156]:
list_Hash2 = []
for i in range(len(y)):
    list_Hash2.append('')
list_Hash2

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',


In [157]:
for i in range(len(y)):
    path = '/content/gdrive/My Drive/Animal Species/' + dico[i]
    #print(path)
    list_Hash2[i] = hash_directory(path)
    print('--hash '+dico[i],'--',list_Hash2[i])

--hash AABI -- ddeab86c870b0614ba3eb793891d54fe232c4efb3121c9d5f6994f1483f79c3c
--hash AABID -- fd2d4757b17dbbb5c7eaa6b08b136118238816f1cfdaacef7daa98cbdd570e49
--hash AABIV -- e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
--hash AALBERG -- 4614ed33e854e5d09c191ea975c89ded298a2b9ea60ceedd55fdc44b6108b79e
--hash AAMARA -- c5348ccbb5811c81fc4af40f1ed797e980e15b82da1f5fddd56c301dd285abe8
--hash AARAB -- 79adfe0e9c3de126179c2c3ff2279f4bc2a6ecec366c7d7a8a2e780bc4391730
--hash AARNINK -- cc8c050946c63c044dad30ce38ca805d7b13b15a8c56da8bddd37ac003c8476b
--hash AARON -- ca820cb14d9294d7f943ba69ad50c3a2f0447bfef57ad7308aa268f87ec83d65
--hash AARONG -- d11ea5347dfe095be2a4e34c571bf8577898fcb25603640524eb3475b03f9618
--hash AARRAS -- e7363f8fbb24cb66fc153720a2489b2c434813ba53bd6f0eec0d537c8bf56c47
--hash AATAR -- 1805345148f0b2298a98b42f5b89820d35f0f92b03a36a9ab0f09f8f4e967513
--hash AATIF -- 28f8c0179dd18b567738e76000223962fbbda23f47a432e1f20f69436361df99
--hash AATZ -- 6e7de77

In [158]:
msg_secret = []
for i in range(len(y)):
    if(list_Hash1[i] != list_Hash2[i]):
        msg_secret.append(1)
    else:
        msg_secret.append(0)

In [159]:
msg_secret

[0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0]

In [160]:
len(msg_secret)

226

In [161]:
np.array(msg_secret, dtype = 'i')

array([0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 0], dtype=int32)

In [162]:
decoded = qc_ldpc.decode(np.copy(np.array(msg_secret, dtype = 'i')))
decoded = qc_ldpc.get_message(decoded)
print(decoded)
print(len(decoded))

[0 1 0 1 0 0 0 0 1 1 1 0 0 1 0 0 1 0 0 1 0 0 0 1 0 1 0 1 0 1 0 0 1 1 1 0 1
 1 1 1 0 1 1 1 0 0 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 1 0 0
 1 0 0 0 1 0 1 1 1 0 1 0 0 1 1 1 0 0 1 0 1 1 1 0 1 1 1 0 1 0 1 0 1 0 1 1 1
 0 1]
113


In [163]:
word

array([0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1])

In [164]:
try:
    assert (msg_secret == y).all()
    assert (decoded == word).all()
except AssertionError:
    print("The secret message must be forwarded")

# #Code script for time evaluation

In [217]:
import time

start = time.time()

import numpy as np

from utils import gaussjordan

from LinearCode import LinearCode


class QC_LDPC(LinearCode):
    """
    Quasi-Cyclic LDPC code representation (extends LinearCode)

    ...

    Methods
    -------
    from_params(n, p, w)
        Init QC-LDPC by length, circulant size and code weight

    _get_circulant_block(polynom)
        Get circulant (p, p) for given vector of size p
        
    """

    def __init__(self, G, H):
        super().__init__(G, H)
        
    @classmethod
    def from_params(cls, n, p, w):
        assert n % p == 0, "p must be delimeter of n"
        
        n0 = n // p
        assert w > 2*n0, "not enough code weight"
        
        fine = False
        
        while not fine:
            blocks = []
            inverse_block = None
            inverse_block_position = None
            
            vector = [1 for _ in range(w)] + [0 for _  in range(n - w)]
            vector = np.array(vector, dtype=int)
            np.random.shuffle(vector)
            
            for i in range(n0):
                circ = vector[i*p:(i+1)*p]
                
                if sum(circ) < 2:
                    inverse_block = None
                    break
        
                block = QC_LDPC._get_circulant_block(circ)
                blocks.append(block)
                    
                A, P = gaussjordan(block, True)
                A = np.array(A, dtype=int)
                P = np.array(P, dtype=int)
                
                if (A == np.eye(p, dtype=int)).all():
                    inverse_block_position = i
                    inverse_block = P
                
            # continue only if inverse circulant found
            fine = True if inverse_block is not None else False
        
        # put inverse block on last position
        blocks[inverse_block_position], blocks[n0-1] = blocks[n0-1], blocks[inverse_block_position]
        H = np.concatenate(blocks, axis=1)

        for i in range(n0):
            blocks[i] = blocks[i] @ inverse_block % 2
            blocks[i] = blocks[i].T            

        Ht = np.concatenate(blocks[:n0-1], axis=0)
        G = np.concatenate((np.eye(Ht.shape[0], dtype=int), Ht), axis=1)
        
        assert (G @ H.T % 2 == 0).all(), "G is not correspond to H"
        
        return cls(G, H)
    
    @staticmethod
    def _get_circulant_block(polynom):
        N = len(polynom)
        block = np.empty((N, N), dtype=int)
        
        for i in range(N):
            block[i] = np.roll(polynom, i)
            
        return block



# EXAMPLE USAGE:

n = 16
p = 8
w = 7

qc_ldpc = QC_LDPC.from_params(n, p, w)
print(qc_ldpc.getG())
print(qc_ldpc.getG().shape)

print(qc_ldpc.getH())
print(qc_ldpc.getH().shape)

word = np.random.randint(2, size=qc_ldpc.getG().shape[0])



word

print(len(word))

encoded = qc_ldpc.encode(word)
print(encoded)
print(len(encoded))

# error vector size n with t or less errors
errors_num = 2
e = [1 for _ in range(errors_num)] + [0 for _  in range(n - errors_num)]
e = np.array(e)
np.random.shuffle(e)
print(e)
print(len(e))

corrupted = (encoded + e) % 2
print(corrupted)
print(len(corrupted))

import os
import shutil

os.chdir('/content/gdrive/My Drive/Animal Species')

#print(ls)

value = 0 
dico = {}
for filename in os.listdir('/content/gdrive/My Drive/Animal Species'):
        if(os.path.isdir(filename)):
            dico[value]  = filename
            value = value + 1
        
        
for i in dico.items():
    print(i)

y = corrupted
print(y)
len(y)

i = 0
while i < len(y):
    my_path = '/content/gdrive/My Drive/Animal Species/' + dico[i]
    os.chdir(my_path)
    if os.path.isfile(my_path + '/file.txt'):
        os.remove(my_path + '/file.txt')
        #print('-- folder f'+str(j),' -- file0 -- ',filename)
    i = i + 1

lis = []
i = 0

while i < len(y):
    print(dico[i])
    count = 0
    for filename in os.listdir('/content/gdrive/My Drive/Animal Species/' + dico[i]):
        print(filename)
        count  = count + 1
    lis.append(count)
    print("-- nombre de fichiers  = ", lis[i],"--")
    print("\n")
    i = i + 1

print(lis)

import hashlib
import os


def hash_directory(path):
    digest = hashlib.sha256()

    for root, dirs, files in os.walk(path):
        for names in files:
            file_path = os.path.join(root, names)

            # Hash the path and add to the digest to account for empty files/directories
            digest.update(hashlib.sha1(file_path[len(path):].encode()).digest())

            # Per @pt12lol - if the goal is uniqueness over repeatability, this is an alternative method using 'hash'
            # digest.update(str(hash(file_path[len(path):])).encode())

            if os.path.isfile(file_path):
                with open(file_path, 'rb') as f_obj:
                    while True:
                        buf = f_obj.read(1024 * 1024)
                        if not buf:
                            break
                        digest.update(buf)

    return digest.hexdigest()

list_Hash1 = []
for i in range(len(y)):
    list_Hash1.append('')
list_Hash1

for i in range(len(y)):
    path = '/content/gdrive/My Drive/Animal Species/' + dico[i]
    #print(path)
    list_Hash1[i] = hash_directory(path)
    print('--hash '+dico[i],'--',list_Hash1[i])

print(y)

liste = ['Balistidae', 'Belonidae', 'Enoploteuthidae', 'Cyprinidae', 'Nephropidae', 'Axiidae', 'Serranidae', 'Cardiidae', 'Scombridae', 'Sparidae', 'Acanthuridae', 'Limidae', 'Sergestidae', 'Acipenseridae', 'Acropomatidae', 'Holothuriidae', 'Squalidae', 'Octopodidae', 'Pectinidae', 'Serranidae']

i = 0
while i < len(y):
    number = np.random.randint(0,20)
    file = '/'+liste[number]+'.txt'
    if(y[i] == 1):
        my_path = 'C:/Users/Willy/Desktop/github/Animal Species/' + dico[i]
        os.chdir(my_path)
        #print(my_path + file)
        if not os.path.exists(my_path + file):
            with open(my_path + file,'w') as fp:
                pass
        else:
            descriptor = open(my_path + file,'a')
            descriptor.write("Species")
            descriptor.close()
        #print('-- folder f'+str(j),' -- file0 -- ',filename)
    i = i + 1

lis = []
i = 0

while i < len(y):
    print(dico[i])
    count = 0
    for filename in os.listdir('/content/gdrive/My Drive/Animal Species/' + dico[i]):
        print(filename)
        count  = count + 1
    lis.append(count)
    print("-- nombre de fichiers  = ", lis[i],"--")
    print("\n")
    i = i + 1



list_Hash2 = []
for i in range(len(y)):
    list_Hash2.append('')
list_Hash2

for i in range(len(y)):
    path = '/content/gdrive/My Drive/Animal Species/' + dico[i]
    #print(path)
    list_Hash2[i] = hash_directory(path)
    print('--hash '+dico[i],'--',list_Hash2[i])

msg_secret = []
for i in range(len(y)):
    if(list_Hash1[i] != list_Hash2[i]):
        msg_secret.append(1)
    else:
        msg_secret.append(0)

print(np.array(msg_secret, dtype = 'i'))


decoded = qc_ldpc.decode(np.copy(np.array(msg_secret, dtype = 'i')))
decoded = qc_ldpc.get_message(decoded)
print(decoded)
print(len(decoded))

print(word)

try:
    assert (msg_secret == y).all()
    assert (decoded == word).all()
except AssertionError:
    print("The secret message must be forwarded")



end = time.time()
print('time = ',end - start)

[[1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1]]
(8, 16)
[[0 0 1 1 1 0 0 1 0 1 0 1 1 0 0 0]
 [1 0 0 1 1 1 0 0 0 0 1 0 1 1 0 0]
 [0 1 0 0 1 1 1 0 0 0 0 1 0 1 1 0]
 [0 0 1 0 0 1 1 1 0 0 0 0 1 0 1 1]
 [1 0 0 1 0 0 1 1 1 0 0 0 0 1 0 1]
 [1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 0]
 [1 1 1 0 0 1 0 0 0 1 1 0 0 0 0 1]
 [0 1 1 1 0 0 1 0 1 0 1 1 0 0 0 0]]
(8, 16)
8
[0 0 0 0 1 1 0 1 0 1 0 0 1 1 1 0]
16
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1]
16
[0 0 0 1 1 1 0 1 0 1 0 0 1 1 1 1]
16


FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: '/content/gdrive/My Drive/Animal Species'